In [2]:
from crossword_solver.candidate_search_helpers import *
from crossword_solver.crossword_helpers import Crossword, load_crossword, Hint, Direction
from crossword_solver.solver_helpers import solving_algorithm, display_results
import pandas as pd
from gensim.models import KeyedVectors

In [ ]:
# Load word2vec
#word2vec_model = KeyedVectors.load_word2vec_format('../data/lemmas.cbow.s100.w2v.bin', binary=True)

In [ ]:
# Read in WordNet contents
wn_voc = pd.read_parquet('../data/wordnet_contents.parquet')

In [4]:
# Read in crossword
cw = load_crossword('../data/crossword1.pickle')

In [ ]:
def search_candidates(hint):
    hint_variations = [hint.hint] # Later each hint should have a list inside already 
    length = hint.length
    candidate_list = []
    for text in hint_variations:
        lemmatised_text = lemmatise_text(text)
        candidate_list.extend(wikipedia_search(lemmatised_text))
        candidate_list.extend(wordnet_search(lemmatised_text))
        candidate_list.extend(create_abbreviated_candidate(text, length))
        candidate_list.extend(create_word2vec_candidates(lemmatised_text))
    clean_candidates(candidate_list)
    candidate_list = remove_duplicates(candidate_list)
    candidate_list = expand_candidates(candidate_list)
    candidate_list = remove_duplicates(candidate_list)
    sorted_candidates = sorted(candidate_list, key = lambda x: x.weight, reverse = True)
    return sorted_candidates

In [7]:
hint = cw.hints[1]
hint

must-valge loom

In [8]:
search_candidates(hint)

[Candidate(text='ämblik', source='word2vec', weight=0.7),
 Candidate(text='imetaja', source='word2vec', weight=0.7),
 Candidate(text='kiskja', source='word2vec', weight=0.7),
 Candidate(text='sisalik', source='word2vec', weight=0.7),
 Candidate(text='olend', source='word2vec', weight=0.7),
 Candidate(text='putukas', source='word2vec', weight=0.7),
 Candidate(text='väliselt', source='word2vec', weight=0.7),
 Candidate(text='küülik', source='word2vec', weight=0.7),
 Candidate(text='olevus', source='word2vec', weight=0.7),
 Candidate(text='elusolend', source='word2vec', weight=0.7),
 Candidate(text='naisekeha', source='word2vec', weight=0.7),
 Candidate(text='koduloom', source='word2vec', weight=0.7),
 Candidate(text='ahv', source='word2vec', weight=0.7),
 Candidate(text='madu', source='word2vec', weight=0.7),
 Candidate(text='isane', source='word2vec', weight=0.7),
 Candidate(text='loomake', source='word2vec', weight=0.7),
 Candidate(text='mustvalge', source='word2vec', weight=0.7),
 Can

In [ ]:
WORDNET_VOCAB = 'wordnet vocab'
WORDNET_VOCAB_WEIGHT = 0.8
def find_regex_match(hint_text, length, wn_list):
    if "..." not in hint_text:
        return []
    matching_group = f"(.{{{length}}})"
    pattern ="^"+ hint_text.replace("...", matching_group)+ "$"
    matches = []
    for w in wn_list:
        match = re.search(pattern, w)
        if match is not None:
            matches.append(match.group(1))
    if len(matches) > 0:
        return convert_results_to_candidates(matches, tag = WORDNET_VOCAB, weight = WORDNET_VOCAB_WEIGHT)
    return []

In [ ]:
hint.hint = 'kilo...'
hint.length = 5
find_regex_match(hint.hint, hint.length, wn_voc['word'])